### [ 데이터 전용 Dataset/DataLoader ]
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset    : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치크기만큼 데이터를 추출해 주는 역할
- 데이터셋 분리 => random_split() 함수 제공 : 타겟 클래스 고려하지 않은 랜덤한 데이터 분리
- 층화기법 => train_test_split() 함수 : scikit-learn의 함수

[1] 모듈로딩 및 데이터 준비<hr>

In [2]:
# [1-1] 모듈로딩
import torch                                            # 텐서 및 수치과학 함수들 관련 모듈
from torch.utils.data import Dataset, DataLoader        # pytorch의 데이터 로딩
from torch.utils.data import Subset                     # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split
import pandas as pd


In [3]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

## [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)


[2] 데이터 로딩 <hr>

In [4]:
## --------------------------------------------------------------------------------
## [2-2] 커스텀 데이터셋 클래스 정의
## --------------------------------------------------------------------------------
## 클래스이름 : ClfDataset
## 부모클래스 : Dataset
## 오버라이딩 : _ _init_ _(self)        : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
##            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
##            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
##                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
## --------------------------------------------------------------------------------
class ClfDataset(Dataset):

    ##- 피쳐와 타겟 저장 및 기타 속성 초기화 
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    ##- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0] 
    
    ##- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS



In [14]:
## --------------------------------------------------------------------------------
## [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
## --------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)
testDS  = ClfDataset(testDF)


print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')
print(trainDF[trainDF.columns[0]].value_counts())

allDS : 60000,  testDS : 10000
0
1    6742
7    6265
3    6131
2    5958
9    5949
0    5923
6    5918
8    5851
4    5842
5    5421
Name: count, dtype: int64


In [15]:
## [2-2-2] 학습용/검증용/테스트용 데이터셋 분리
## 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
## 검증용   : 제대로 데이텃에서 규칙/패턴을 찾는지 확인 용도
##           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
## 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

## 학습용 데이터셋의 개수 
print(f'allDS :{len(allDS)}개, testDS : {len(testDS)}개')

## 학습용 데이터셋에서 타겟/라벨만 추출 
targetList = allDS.y 
print(f'targetList => \n{targetList}')

## 학습용 데이터셋에서 타겟/라벨만 추출 
dataNP = allDS.y.shape
print(f'dataNP => \n{dataNP}')

## 학습용 데이터셋에서 타겟/라벨 인덱스 생성
dataIndexList =list( range(len(allDS)))


allDS :60000개, testDS : 10000개
targetList => 
[5 0 4 ... 5 6 8]
dataNP => 
(60000,)


In [16]:
## 학습용/검증용 데이터셋 인덱스 분리
## -train_test_split() 함수 : train:test = 75:25 비율로 학습용, 테스트용 데이터셋 분리
##                            stratify : 분류용 데이터셋 경우 카테고리 비율 유지해서
##                                       데이터셋 분리
X_trainIdx, X_validIdx , y_train, y_valid = train_test_split( dataIndexList,
                                                              targetList,
                                                              train_size=0.8,
                                                              stratify=targetList,
                                                              random_state=10 )
print(f'X_trainIdx : {len(X_trainIdx)}개, {type(X_trainIdx)}')
print(f'X_validIdx : {len(X_validIdx)}개, {type(X_validIdx)}')
print(f'testDS     : {len(testDS)}개')

X_trainIdx : 48000개, <class 'list'>
X_validIdx : 12000개, <class 'list'>
testDS     : 10000개


In [17]:
## -----------------------------------------------------------------
## 학습용/검증용 데이터셋 생성 ===> Dataset ==> 2개 Subset 분리
## -----------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f'allDS   : {type(allDS)},   {len(allDS)}개')
print(f'trainDS : {type(trainDS)}, {len(trainDS)}개')
print(f'validDS : {type(validDS)}, {len(validDS)}개')

allDS   : <class '__main__.ClfDataset'>,   60000개
trainDS : <class 'torch.utils.data.dataset.Subset'>, 48000개
validDS : <class 'torch.utils.data.dataset.Subset'>, 12000개


In [18]:
## [2-2-3] 학습용/검증용/테스트용 데이터셋 속성
## - datasets 타입 속성
## - data           속성 : 실제 이미지의 로우 데이터 즉, ndarray
## - targets        속성 : 실제 타겟 로우 데이터 리스트 list
## - classes        속성 : 타겟의 종류 
## - class_of_idx   속성 : 타겟의 수치화 
print(f'{type(testDS)}------------')
print(testDS.y.shape, 
      len(testDS.y), sep='\n')

## - dataset.Subset타입 속성
## - dataset                  속성 : 쪼개지기전의 원본 데이터셋 정보 확인
## - indices                  속성 : 선택된 데이터의 인덱스 정보 
## - dataset.data             속성 : 실제 이미지의 로우 데이터 즉, ndarray
## - dataset.targets          속성 : 실제 타겟 로우 데이터 리스트 list
## - dataset.classes          속성 : 타겟의 종류 
## - dataset.class_of_idx     속성 : 타겟의 수치화 
print(f'\n{type(trainDS)}------------')
print(trainDS.dataset, 
      trainDS.indices,
      trainDS.dataset.y.shape,
      len(trainDS.indices),
      sep='\n\n')

print(f'\n{type(validDS)}------------')
print(validDS.dataset, 
      validDS.indices,
      validDS.dataset.y.shape,
      len(validDS.indices),
      sep='\n\n')

<class '__main__.ClfDataset'>------------
(10000,)
10000

<class 'torch.utils.data.dataset.Subset'>------------

[50851, 55313, 26874, 1907, 21659, 17615, 733, 15573, 24172, 24292, 30684, 28841, 40676, 6739, 8470, 48508, 6161, 53187, 11472, 10208, 33180, 21570, 58366, 36158, 33237, 22149, 10905, 48539, 9692, 5666, 39527, 7822, 25134, 35241, 10616, 51532, 14456, 50843, 53034, 122, 10836, 1212, 23607, 8719, 24530, 31693, 6134, 10405, 28285, 6463, 54937, 12480, 26666, 25233, 20645, 30189, 40357, 37673, 30897, 46854, 16503, 26075, 204, 46618, 36813, 44066, 12493, 24623, 59151, 27783, 6522, 57097, 24932, 13101, 29862, 52231, 18974, 46339, 23857, 36531, 33742, 2088, 52674, 27593, 48048, 5290, 44209, 11841, 37752, 54259, 38064, 56470, 51049, 57091, 16018, 11010, 20735, 49100, 48642, 14071, 14757, 38130, 18835, 35431, 13468, 1483, 47034, 17750, 15567, 11350, 1427, 3344, 48377, 56030, 45126, 25529, 54950, 59725, 32231, 35253, 38737, 50426, 43834, 22559, 34115, 3619, 4761, 34863, 43158, 43943, 1

In [23]:
## -------------------------------------------------------------------
## 학습용/검증용/테스트용 데이터셋에 카테고리별 데이터 분포
## - 균형 데이터셋 & 불균형 데이터셋 
## -------------------------------------------------------------------
from collections import Counter

# 테스트 데이터셋의 카테고리별 분포
allC=Counter(allDS.y.tolist())
allDict = dict( sorted(allC.items()) )
print("allDict   =>", { f'{k}번' : int((v/len(allDS))*100) for k, v in allDict.items()})

# 학습용 데이터셋의 카테고리별 분포
sel_train=[ trainDS.dataset.y[idx].item() for idx in trainDS.indices]
trainC=Counter(sel_train)

trainDict = dict(sorted(trainC.items()))
print("trainDict =>", { f'{k}번' : int((v/len(trainDS))*100) for k, v in trainDict.items()})

# 검증용 데이터셋의 카테고리별 분포
val_train=[ validDS.dataset.y[idx].item() for idx in validDS.indices]
validC=Counter(val_train)

validDict = dict(sorted(validC.items()))
print("validDict =>", { f'{k}번' : int((v/len(validDS))*100) for k, v in validDict.items()})

allDict   => {'0번': 9, '1번': 11, '2번': 9, '3번': 10, '4번': 9, '5번': 9, '6번': 9, '7번': 10, '8번': 9, '9번': 9}
trainDict => {'0번': 9, '1번': 11, '2번': 9, '3번': 10, '4번': 9, '5번': 9, '6번': 9, '7번': 10, '8번': 9, '9번': 9}
validDict => {'0번': 9, '1번': 11, '2번': 9, '3번': 10, '4번': 9, '5번': 9, '6번': 9, '7번': 10, '8번': 9, '9번': 9}
